# 1a)  IMPORTING AND CLEANING THE DATA

In [1]:
import pandas as pd
import os
import re
import tldextract
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import spatial
import os, time, http.client
from termcolor import colored
import sys
import json
from pygments import highlight, lexers, formatters
import pycountry

In [ ]:
#iclabcombined = pd.read_csv("combined.csv", dtype={'country' : str})

Dropping the columns which were meant for internal use of ICLAB team and the header columns which have come into dataset via join

In [ ]:
# iclabcombined.drop(['filename', 'as_number'], axis=1, inplace= True)

# iclabcombined.drop([ 3078134,  4235577, 12411076, 18967078, 25041685, 26600938,
#             28192530, 29460039, 30795516, 32569776, 34391850, 36093972,
#             37793956, 39298313, 40934970, 42276914, 42976448, 44455181,
#             47068048, 49383978], inplace = True)

Correcting wrongly coded boolean values in the datasets

In [ ]:
# iclabcombined['dns'].replace('false',False,inplace=True)
# iclabcombined['dns_all'].replace('false',False,inplace=True)
# iclabcombined['block'].replace('false',False,inplace=True)
# iclabcombined['packet_updated'].replace('false',False,inplace=True)
# iclabcombined['censored_updated'].replace('false',False,inplace=True)

# iclabcombined['dns'].replace('true',True,inplace=True)
# iclabcombined['dns_all'].replace('true',True,inplace=True)
# iclabcombined['block'].replace('true',True,inplace=True)
# iclabcombined['packet_updated'].replace('true',True,inplace=True)
# iclabcombined['censored_updated'].replace('true',True,inplace=True)

# 1b) Importing categories

In [ ]:
# catDF = pd.read_fwf('type_domain_mapping.txt',header=None, names=[ "Name"], sep="")

In [ ]:
# catDF = catDF.Name.apply(lambda x: pd.Series(str(x).split("|||"))).rename(columns={0:'category',1:'site'})

In [ ]:
# allCategories = set(catDF['category'])
# len(allCategories)

In [ ]:
# ADDING CATEGORY AS A PERMANENT COLUMN

# urlListAll = iclabcombined['url'].tolist()
# sitesListAll = []

# for item in urlListAll:
#     domainAll = tldextract.extract(item)[1]
#     suffixAll = tldextract.extract(item)[2]
#     sitesListAll.append(domainAll + "." + suffixAll)

In [ ]:
# iclabcombined['site'] = sitesListAll
# iclabcombined = pd.merge(iclabcombined, catDF, on='site',how='left')

In [ ]:
# PICKLING OUR DATA FOR FASTER ACCESS
#iclabcombined.to_pickle('iclabcombined.pkl')

==
LOADING DATA FRAME FROM PICKLE==

In [439]:
iclabcombined = pd.read_pickle('iclabcombined.pkl')

In [440]:
print("The number of observations in our original data set are {0} and the number of features/columns are {1} ".format(iclabcombined.shape[0],iclabcombined.shape[1]))

The number of observations in our original data set are 50436846 and the number of features/columns are 17 


# 2. ANALYZING FALSE POSITIVES 

In [ ]:
# #function to find blocked URLs
# def blockedURLs(cc):
#     return (set(iclabcombined[ (iclabcombined['country']== cc)& ((iclabcombined['censored_updated']==True)) ]['url']))

### 2a) Looking at sites which only have censored_updated = True

The below query produces 167 urls, these are the URLs for which the ICLab team believes the packets might have been injected i.e. becuase there was a response after RST or FIN or maybe because of ICMP admin prohibited i.e. they determined there's a firewall that blocks a connection. Removing these from our data as these are cases of PROBABLE CENSORSHIP and not ACTUAL censorship

In [ ]:
#( set(iclabcombined[ (iclabcombined['country']=='US')  & ((iclabcombined['dns']== False) & (iclabcombined['dns_all']== False) & (iclabcombined['block']== False) & (iclabcombined['packet_updated']== True))]['url']))

Even urls for which a 4xx or 5xx result was returned are accessible and some of them are URLS commonly accessed in US

In [ ]:
# set(iclabcombined
#     [ (iclabcombined['country']=='US') 
#                    & 
#      (
#          (iclabcombined['http_status']== 403.0) | (iclabcombined['http_status']== 404.0) | (iclabcombined['http_status']== 504.0)
#      )
#      &
#      (   
#                        (iclabcombined['dns']== False) & (iclabcombined['dns_all']== False)
#                        & (iclabcombined['block']== False) & (iclabcombined['dns_all']== False) 
#                        & (iclabcombined['packet_updated']== True)
#                    )]
#      ['url'])

In [ ]:
#httpError = np.arange(400,600,1,dtype=float)

The falsePositIndex here are the sites which return a http 200 and only show packet_updated = True. In below query, I eliminate the countries where the result of HTTP was not a 400/500 errorand the only mechanism of censorship was that the packet_updated = True i.e. all these are probable censorships

In [ ]:
# falsePositIndex = iclabcombined[
#     (~iclabcombined.http_status.isin(httpError))  
#                    & (   (iclabcombined['dns']== False) & (iclabcombined['dns_all']== False) & (iclabcombined['block']== False) & (iclabcombined['dns_all']== False) & (iclabcombined['packet_updated']== True)
#                      )] [['country','url','http_status']].index.tolist()

In [ ]:
# print("We have " + str(len(falsePositIndex)) + " probable censoriships which could be false positives")

In [ ]:
# iclabcombined.drop(falsePositIndex,inplace=True)

Now having a look at FALSE NEGATIVES
We define false negatives as websites which are showing censored_updated = False here but upon testing actually are inaccessible

In [ ]:
# set(iclabcombined[(iclabcombined['country']=='US')&(iclabcombined['censored_updated']==False) & (iclabcombined.http_status.isin([403.0]))]['url'])

In total as seen below we have 3389 websites which show up as not censored in US but the corresponding HTTP results are error codes, manual testing of these websites show that they are accessible, its not possible to actually determine the status of all false negatives. Based on our testing and seeing that they are not marked as blocked eventually and also because we believe that false negatives are not too critical to our project  $hence$ $we$ $wont$ $drop$ $them$ $from$ $the$ $data$ $set$

## 2b) Removing the FALSE POSITIVIES (US blocked sites) from our main dataset iclabcombined

From this data looking at the sites that are shown as censored in US. THESE SITES HAVE CENSORED = TRUE AND RETURN A HTTP ERROR

In [ ]:
#set( iclabcombined  [  (iclabcombined['country']=='US') & (iclabcombined.http_status.isin(httpError) & ( (iclabcombined['dns_all']== True) | (iclabcombined['block']== True)|(iclabcombined['censored_updated']== True) | (iclabcombined['packet_updated']== True)))]['url'])

In [ ]:
# SITES SHOWING HTTP status reason as block in US but still accessible
#set( iclabcombined  [  (iclabcombined['country']=='US') & (iclabcombined.http_reason.str.contains('block'))]['url'])

Overall all the above results show a very dirty data, we basically removed the root urls of all the sites that show 404 in US and then checked how many of them are accesible. There are no patterns while some are showing as blocked on testing others are not. BELOW WE REMOVE THESE FALSE POSITIVIES FROM THE US

In [ ]:
USFPlist = (iclabcombined [(iclabcombined['country']=='US') & (iclabcombined['censored_updated'] == True)]['site']).unique().tolist()

In [ ]:
USblockIclab = len(iclabcombined[iclabcombined['site'].isin(USFPlist)])

In [ ]:
print("There are {} unique blocked websites in the US".format(len(USFPlist)))
print("There are {} total observations of these blocked websites throughout our dataset".format(USblockIclab))

In [ ]:
# USFPIndex is the index of all those observations for the unique sites which are blocked in US and we are going to remove from the entire DB
USFPIndex = iclabcombined[iclabcombined['site'].isin(USFPlist)].index

# iclabCLEAN is our cleaned dataframe

In [ ]:
iclabCLEAN = iclabcombined.drop(USFPIndex)

## PICKLING OUR CLEAN DATASET FOR FASTER ACCESS

In [ ]:
#iclabCLEAN.to_pickle('iclabCLEAN.pkl')

In [441]:
# LOADING DATA FRAME FROM PICKLE
iclabCLEAN = pd.read_pickle('iclabCLEAN.pkl')

In [442]:
# COMPARNING DATA SIZES of RAW and CLEAN DATA 
print("The number of observations in our original data set are {0} and the number of features/columns are {1} ".format(iclabcombined.shape[0],iclabcombined.shape[1]))
print("The number of observations in our CLEANED data set i.e. without sites blocked in USA are {0} and the number of features/columns are {1} ".format(iclabCLEAN.shape[0],iclabCLEAN.shape[1]))

The number of observations in our original data set are 50436846 and the number of features/columns are 17 
The number of observations in our CLEANED data set i.e. without sites blocked in USA are 40303928 and the number of features/columns are 17 


$******************************************************************$

# 3.  EXPERIMENT 1

Looking at the exact webpages blocked

In [ ]:
# blockpageTR = set(iclabCLEAN[(iclabCLEAN['country']=='TR') & ((iclabCLEAN['dns_all']==True)| (iclabCLEAN['censored_updated']==True) | (iclabCLEAN['block']==True) | (iclabCLEAN['packet_updated']==True)) ]['url'])
# len(blockpageTR)

In [ ]:
# blockpageRU = set(iclabCLEAN[(iclabCLEAN['country']=='RU') & ((iclabCLEAN['dns_all']==True)| (iclabCLEAN['censored_updated']==True) | (iclabCLEAN['block']==True) | (iclabCLEAN['packet_updated']==True)) ]['url'])
# len(blockpageRU)

In [ ]:
# RUblockpageTR = blockpageTR.intersection(blockpageRU)
# RUblockpageTR

# EXPERIMENT 2

In [ ]:
# blockIN = set(iclabCLEAN[(iclabCLEAN['country']=='IN') & ((iclabCLEAN['censored_updated']==True))]['site'])
# len(blockIN)

In [ ]:
# blockCN = set(iclabCLEAN[(iclabCLEAN['country']=='CN') & ((iclabCLEAN['censored_updated']==True))]['site'])
# len(blockCN)

In [ ]:
# blockVN = set(iclabCLEAN[(iclabCLEAN['country']=='VN') & ((iclabCLEAN['censored_updated']==True))]['site'])
# len(blockVN)

In [ ]:
# blockRU = set(iclabCLEAN[(iclabCLEAN['country']=='RU') & ((iclabCLEAN['dns_all']==True)| (iclabCLEAN['censored_updated']==True)| (iclabCLEAN['block']==True) | (iclabCLEAN['packet_updated']==True))]['site'])
# len(blockRU)

In [ ]:
# blockTR = set(iclabCLEAN[(iclabCLEAN['country']=='TR') & ((iclabCLEAN['dns_all']==True)| (iclabCLEAN['censored_updated']==True) | (iclabCLEAN['block']==True) | (iclabCLEAN['packet_updated']==True)) ]['site'])
# len(blockTR)

In [ ]:
# blockNL = set(iclabCLEAN[(iclabCLEAN['country']=='NL') & ((iclabCLEAN['censored_updated']==True))]['site'])
# len(blockNL)

In [ ]:
# blockNZ = set(iclabCLEAN[(iclabCLEAN['country']=='NZ') & ((iclabCLEAN['censored_updated']==True))]['site'])
# len(blockNZ)

In [ ]:
# AU_NZ = blockNZ.intersection(blockAU)
# len(AU_NZ)

In [ ]:
# blockMX = set(iclabCLEAN[(iclabCLEAN['country']=='MX') & ((iclabCLEAN['censored_updated']==True))]['site'])
# len(blockMX)

In [ ]:
# IN_RU = blockIN.intersection(blockRU)
# (IN_RU)

Intersections as per EXPERIMENT 2

In [ ]:
# RU_TR = blockRU.intersection(blockTR)

In [ ]:
# BR_MX = blockMX.intersection(blockBR)

In [ ]:
# NL_TR = blockNL.intersection(blockTR)

As part of Experiment 2 : 
(a) A look at censored FB Pages in India

Looking at worldwide distribution of FB blocking

In [ ]:
# iclabCLEAN[iclabCLEAN['url'].str.contains('facebook')].groupby('country').count()

Looking at FB pages censored in India

In [ ]:
# fbIndBlock = set (iclabcombined [ ( (iclabcombined['block']== True) | (iclabcombined['censored_updated']== True) | (iclabcombined['packet_updated']== True) ) & (iclabcombined['url'].str.contains('facebook')) & (iclabcombined['country']=='IN') ]['url'])
# fbIndBlock

(b) A look into religious censorship in the world

In [ ]:
# iclabCLEAN[ (iclabCLEAN['category'].isin(['Global Religion','Alternative Beliefs'])) & ( (iclabCLEAN['block']== True) | (iclabCLEAN['censored_updated'] == True) ) ].groupby(['country', 'site']).first()

# 4) EXPERIMENT 3 : 

Defining a function which computes contribution of each category in a countries blocked sites

In [3]:
def Category_contributions(cc):
    catDataCC = iclabCLEAN[(iclabCLEAN['country']==cc) & (iclabCLEAN['censored_updated']==True) ]
    catDataCC= catDataCC.drop(['server_t', 'censored_updated','site','dns_all','packet_reason','url','dns','dns_reason','dns_reason_all','schedule_name','http_status','block','body_len','http_reason','packet_updated'], axis=1)
    catDataCC = catDataCC.groupby('category').count()
    catDataCC['%Cat'] = (catDataCC['country']*100)/(catDataCC['country'].sum())
    catDataCC= catDataCC.reset_index().drop('country',axis=1)
    catDataCC= catDataCC.rename(columns={'country':'Number of sites in' + str(cc),'%Cat':cc})
    return catDataCC

In [ ]:
# INplot = Category_contributions('IN')
# display(INplot)
# TRplot =Category_contributions('TR')
# RUplot = Category_contributions('RU')
# KRplot= Category_contributions('KR')
# USplot = Category_contributions('US')
# CNplot = Category_contributions('CN')
# CAplot  = Category_contributions('CA')
# ZAplot  = Category_contributions('ZA')

 Plotting the graph below:

In [ ]:
# graphcompare1 = pd.merge(INplot,TRplot, on='category', how='outer')
# graphcompare2 = pd.merge(graphcompare1,KRplot, on='category', how='outer')
# graphcompare3 = pd.merge(graphcompare2,RUplot, on='category', how='outer')
# graphcompare4 = pd.merge(graphcompare3,USplot, on='category', how='outer')
# graphcompare5 = pd.merge(graphcompare4,CNplot, on='category', how='outer')

In [ ]:
# # set width of bar
# barWidth = 0.15
 
# # set height of bar
# TR = graphcompare3['TR']
# RU = graphcompare3['RU']
# IN = graphcompare3['IN']
# KR = graphcompare3['KR']
 
# # Set position of bar on X axis
# r1 = np.arange(len(TR))
# r2 = [x + barWidth for x in r1]
# r3 = [x + barWidth for x in r2]
# r4 = [x + barWidth for x in r3]
 
# # Make the plot
# plt.figure(figsize = (20,12))
# plt.bar(r1, TR, color='green', width=barWidth, edgecolor='dimgrey', label='Turkey')
# plt.bar(r2, RU, color='red', width=barWidth ,  edgecolor='dimgrey',label='RU')
# plt.bar(r3, IN, color='blue', width=barWidth,  edgecolor='dimgrey',label='India')
# plt.bar(r4, KR, color='darkorange', width=barWidth,  edgecolor= 'dimgrey',label='South Korea') 

# # Add xticks on the middle of the group bars
# plt.ylabel("Percentage", size=15)
# plt.ylim(0,100)
# plt.xticks([r for r in range(len(TR))], graphcompare3['category'].tolist() ,rotation=-90, size=16)

# # Create legend & Show graphic
# plt.legend(loc = 'upper left', fontsize = 15)
# plt.title("\n Category-wise blocking in countries \n", size = 15)
# plt.show()

A few sample metrics

In [ ]:
# graphcompare5.fillna(0,inplace=True)

In [ ]:
# KRvals = graphcompare5['KR']
# RUvals = graphcompare5['RU']
# INvals = graphcompare5['IN']
# TRvals = graphcompare5['TR']

In [ ]:
# KRsimilarityRU = (1 - spatial.distance.cosine(KRvals, RUvals))*100
# KRsimilarityRU

In [ ]:
# INsimilarityTR = (1 - spatial.distance.cosine(INvals, TRvals))*100
# INsimilarityTR

In [ ]:
# INsimilarityKR = (1 - spatial.distance.cosine(INvals, KRvals))*100
# INsimilarityKR

In [ ]:
# INsimilarityRU = (1 - spatial.distance.cosine(INvals, RUvals))*100
# INsimilarityRU

In [ ]:
# KRsimilarityTR = (1 - spatial.distance.cosine(KRvals, TRvals))*100
# KRsimilarityTR

In [ ]:
# RUsimilarityTR = (1 - spatial.distance.cosine(RUvals, TRvals))*100
# RUsimilarityTR


# 5) SIMILARITY METRIC DATAFRAME FOR ALL COUNTRIES

In [443]:
allCountries = (iclabCLEAN['country'].unique())
print("The total countries we have in our clean dataset are {}".format(len(allCountries)))

The total countries we have in our clean dataset are 55


In [444]:
allCategories = (iclabCLEAN['category'].unique())
print("The total unique categories we have in our clean dataset are {}".format(len(allCategories)))

The total unique categories we have in our clean dataset are 72


In [235]:
#simMetricDFIndex = (iclabCLEAN['category'].dropna().unique())
simMetricDFIndex = (iclabCLEAN['category'].unique())

In [236]:
simMetricDF = pd.DataFrame(index = simMetricDFIndex, columns=None)
simMetricDF.head()

""
File Sharing and Storage
Reference
Information Technology
Instant Messaging
News and Media


In [238]:
simMetricDF['dummy']= pd.Series(np.zeros(len(simMetricDF)))
simMetricDF.head()

,dummy
File Sharing and Storage,NaN
Reference,NaN
Information Technology,NaN
Instant Messaging,NaN
News and Media,NaN


In [239]:
simMetricDF.reset_index(inplace=True)
simMetricDF.rename(columns={'index':'category'},inplace=True)
simMetricDF.head()

,category,dummy
0,File Sharing and Storage,NaN
1,Reference,NaN
2,Information Technology,NaN
3,Instant Messaging,NaN
4,News and Media,NaN


In [240]:
for country in allCountries:
    simMetricDF = pd.merge(simMetricDF,Category_contributions(country), on= 'category', how='outer')

In [241]:
#simMetricDF = simMetricDF.drop(['base'],axis=1)
simMetricDF.set_index('category',inplace = True)

==>> REPLACING NaN's with 0

In [242]:
simMetricDF.fillna(0,inplace=True)

In [362]:
simMetricDF.drop("dummy",axis=1,inplace = True)

,UA,CO,KR,US,BG,BZ,AU,JP,NL,HK,...,TR,PL,DZ,BR,SC,SA,AT,GR,CR,LI
category,,,,,,,,,,,,,,,,,,,,,
File Sharing and Storage,0.000000,0.0,3.433968,0.0,0.0,0.0,13.953488,0.000000,0.000000,9.090909,...,2.441077,0.0,0.0,0.000000,0.0,0.911029,0.0,0.0,0.0,0.0
Reference,0.000000,0.0,1.981686,0.0,0.0,0.0,0.000000,1.886792,0.000000,9.090909,...,0.841751,0.0,0.0,7.692308,0.0,0.000000,0.0,0.0,0.0,0.0
Information Technology,10.762332,0.0,2.153384,0.0,100.0,0.0,11.627907,7.547170,10.000000,0.000000,...,0.000000,0.0,0.0,7.692308,0.0,1.957744,0.0,0.0,0.0,0.0
Instant Messaging,0.000000,0.0,0.064387,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
News and Media,0.448430,12.5,7.168407,0.0,0.0,50.0,16.279070,15.094340,47.777778,9.090909,...,28.198653,0.0,0.0,23.076923,0.0,9.866253,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Online Meeting,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
Brokerage and Trading,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
Dynamic DNS,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [363]:
#Getting all columns i.e. countries
simCols=[]
for i in range(len(simMetricDF.columns)):
    simCols.append(simMetricDF.columns[i])
print("We have {} unique countries".format(len(simCols)))

We have 55 unique countries including dummy


In [364]:
#Creating pairs to do calculation of spatial distance
from itertools import combinations
pairs = [comb for comb in combinations(simCols, 2)]
print("We have {} unique pairs from these countries including dummy".format(len(pairs)))

We have 1485 unique pairs from these countries including dummy


==>> Mathematical checks

In [365]:
#Both vectors are non zeroes
#print(" **** Similarity is {} ****".format(1-(spatial.distance.cosine(simMetricDF['AU'], simMetricDF['IN']))))

# When even one vector is completeley a zero vector
#print(" **** Similarity is {} ****".format(1-(spatial.distance.cosine(simMetricDF['US'], simMetricDF['IN']))))

In [366]:
#NewSimVal will store the similarity value calculated from the spatial cosine distance formula

NewSimVals = []
for item in pairs:
    if (np.sum(simMetricDF[item[0]]) == 0) & (np.sum(simMetricDF[item[1]]) ==0):
        NewSimVals.append((item,100))
    elif (np.sum(simMetricDF[item[0]]) == 0) | (np.sum(simMetricDF[item[1]]) ==0):
        NewSimVals.append((item,0))
    else :
        NewSimVals.append((item,(1 - spatial.distance.cosine(simMetricDF[item[0]], simMetricDF[item[1]]))*100))

In [367]:
NewSimVals

[(('UA', 'CO'), 0.1593395361172978),
 (('UA', 'KR'), 0.6847792559200427),
 (('UA', 'US'), 0),
 (('UA', 'BG'), 12.093020530687948),
 (('UA', 'BZ'), 0.7125880685231345),
 (('UA', 'AU'), 4.98016634651659),
 (('UA', 'JP'), 8.579139186286355),
 (('UA', 'NL'), 7.330689060535911),
 (('UA', 'HK'), 0.3467911474699381),
 (('UA', 'LT'), 0.22534013299931877),
 (('UA', 'RO'), 0.0),
 (('UA', 'KE'), 0.45068026599864863),
 (('UA', 'ES'), 0.1679586184817783),
 (('UA', 'VN'), 0.0),
 (('UA', 'MX'), 0.0),
 (('UA', 'PE'), 0.3562940342615617),
 (('UA', 'SE'), 0.48093941543784213),
 (('UA', 'NZ'), 2.672205256961746),
 (('UA', 'FR'), 0.1679586184817783),
 (('UA', 'ID'), 0),
 (('UA', 'ZA'), 0.0),
 (('UA', 'HU'), 0.0),
 (('UA', 'SK'), 0.5451210197267242),
 (('UA', 'CL'), 0.0),
 (('UA', 'CN'), 1.421422380505677),
 (('UA', 'LU'), 0.0),
 (('UA', 'IN'), 1.8067619121192702),
 (('UA', 'SG'), 0.5038758554453349),
 (('UA', 'TW'), 7.669772649165408),
 (('UA', 'MD'), 0.3359172369635566),
 (('UA', 'NO'), 0),
 (('UA', 'RU'

In [368]:
simMetricVals = pd.DataFrame(index=pairs, columns = None )
simMetricVals.head()

""
"(UA, CO)"
"(UA, KR)"
"(UA, US)"
"(UA, BG)"
"(UA, BZ)"


In [369]:
simMetricVals['similarity'] = [j for (i,j) in NewSimVals]
simMetricVals.head()

,similarity
"(UA, CO)",0.159340
"(UA, KR)",0.684779
"(UA, US)",0.000000
"(UA, BG)",12.093021
"(UA, BZ)",0.712588


In [370]:
simMetricVals.reset_index(inplace=True)
simMetricVals.head()

,index,similarity
0,"(UA, CO)",0.159340
1,"(UA, KR)",0.684779
2,"(UA, US)",0.000000
3,"(UA, BG)",12.093021
4,"(UA, BZ)",0.712588


In [372]:
simMetricVals.rename(columns={'index':'pair'},inplace=True)
simMetricVals.head()

,pair,similarity
0,"(UA, CO)",0.159340
1,"(UA, KR)",0.684779
2,"(UA, US)",0.000000
3,"(UA, BG)",12.093021
4,"(UA, BZ)",0.712588


In [373]:
simMetricVals[['country_code_alpha2_A', 'country_code_alpha2_B']] = pd.DataFrame(simMetricVals['pair'].tolist(), index=simMetricVals.index)
simMetricVals.head()

,pair,similarity,country_code_alpha2_A,country_code_alpha2_B
0,"(UA, CO)",0.159340,UA,CO
1,"(UA, KR)",0.684779,UA,KR
2,"(UA, US)",0.000000,UA,US
3,"(UA, BG)",12.093021,UA,BG
4,"(UA, BZ)",0.712588,UA,BZ


In [374]:
# Reordering columns

tempCol = simMetricVals['similarity']
simMetricVals.drop(labels=['similarity'], axis=1,inplace=True)
simMetricVals.insert(3, 'similarity', tempCol)
simMetricVals.drop('pair',axis=1,inplace=True)
simMetricVals.head()

,country_code_alpha2_A,country_code_alpha2_B,similarity
0,UA,CO,0.159340
1,UA,KR,0.684779
2,UA,US,0.000000
3,UA,BG,12.093021
4,UA,BZ,0.712588


In [375]:
# Pickling the simMetricVals data set
#simMetricVals.to_pickle("simMetricVals.pkl")

## Reading simMetricVals from pickle

In [422]:
worldMapViz = pd.read_pickle("simMetricVals.pkl")

In [423]:
worldMapViz.head()

,country_code_alpha2_A,country_code_alpha2_B,similarity
0,UA,CO,0.159340
1,UA,KR,0.684779
2,UA,US,0.000000
3,UA,BG,12.093021
4,UA,BZ,0.712588


worldMapDemoDF is the similarity metric data frame without the NaN values
NaN values arise when both the countries have 0 for a particular category and as a result the spatial cosine formula returns an undefined NaN 

In [424]:
# def countryCodeConversion()

inputCountriesA = worldMapViz['country_code_alpha2_A']
inputCountriesB = worldMapViz['country_code_alpha2_B']

countries = {}
for country in pycountry.countries:
    countries[country.alpha_2] = country.alpha_3


alpha3codesA = [countries.get(country, 'Unknown code') for country in inputCountriesA]
alpha3codesB = [countries.get(country, 'Unknown code') for country in inputCountriesB]

In [425]:
worldMapViz['country_code_alpha2_A'] = alpha3codesA
worldMapViz['country_code_alpha2_B'] = alpha3codesB
worldMapViz.rename(columns={'country_code_alpha2_A':'country_code_alpha3_A','country_code_alpha2_B':'country_code_alpha3_B'},inplace=True)

In [426]:
worldMapViz.head()

,country_code_alpha3_A,country_code_alpha3_B,similarity
0,UKR,COL,0.159340
1,UKR,KOR,0.684779
2,UKR,USA,0.000000
3,UKR,BGR,12.093021
4,UKR,BLZ,0.712588


### CONVERTING THIS DATA FRAME INTO JSON

In [433]:
worldDictList = [{i:{l:{"sim":(n)} for l,n in zip(g['country_code_alpha3_B'], g['similarity'])}} for i,g in worldMapViz.groupby('country_code_alpha3_A')] 

In [434]:
worldDictList

[{'AND': {'CAN': {'sim': 0.0},
   'SRB': {'sim': 0.0},
   'FIN': {'sim': 100.0},
   'MYS': {'sim': 100.0},
   'BEL': {'sim': 100.0},
   'ISL': {'sim': 100.0},
   'VEN': {'sim': 100.0},
   'PRT': {'sim': 0.0},
   'DNK': {'sim': 100.0},
   'CZE': {'sim': 100.0},
   'ISR': {'sim': 100.0},
   'TUR': {'sim': 0.0},
   'POL': {'sim': 100.0},
   'DZA': {'sim': 100.0},
   'BRA': {'sim': 0.0},
   'SYC': {'sim': 0.0},
   'SAU': {'sim': 0.0},
   'AUT': {'sim': 0.0},
   'GRC': {'sim': 100.0},
   'CRI': {'sim': 100.0},
   'LIE': {'sim': 100.0}}},
 {'AUS': {'JPN': {'sim': 47.37985310765025},
   'NLD': {'sim': 66.53417405518506},
   'HKG': {'sim': 34.88431051590912},
   'LTU': {'sim': 10.200306013770689},
   'ROU': {'sim': 30.41143685078822},
   'KEN': {'sim': 27.200816036721843},
   'ESP': {'sim': 17.740004829626464},
   'VNM': {'sim': 0.0},
   'MEX': {'sim': 21.504133222818812},
   'PER': {'sim': 43.00826644563763},
   'SWE': {'sim': 21.770330683015526},
   'NZL': {'sim': 60.48037468917792},
   'FRA

In [435]:
finalStr = "{"

for item in worldDictList:
    i = (json.dumps(item))
    i = i[1:-1]
    finalStr = finalStr + i
    finalStr = finalStr + ","

finalStr = finalStr[0: -1]
finalStr = finalStr + "}"
jsonFormatted = json.dumps(json.loads(finalStr),indent =4, sort_keys=True)
jsonBeautified = highlight(str(jsonFormatted), lexers.JsonLexer(), formatters.TerminalFormatter())
print(jsonBeautified)

{
    "AND": {
        "AUT": {
            "sim": 0.0
        },
        "BEL": {
            "sim": 100.0
        },
        "BRA": {
            "sim": 0.0
        },
        "CAN": {
            "sim": 0.0
        },
        "CRI": {
            "sim": 100.0
        },
        "CZE": {
            "sim": 100.0
        },
        "DNK": {
            "sim": 100.0
        },
        "DZA": {
            "sim": 100.0
        },
        "FIN": {
            "sim": 100.0
        },
        "GRC": {
            "sim": 100.0
        },
        "ISL": {
            "sim": 100.0
        },
        "ISR": {
            "sim": 100.0
        },
        "LIE": {
            "sim": 100.0
        },
        "MYS": {
            "sim": 100.0
        },
        "POL": {
            "sim": 100.0
        },
        "PRT": {
            "sim": 0.0
        },
        "SAU": {
            "sim": 0.0
        },
        "SRB": {
            "sim": 0.0
        },
        "SYC": {
            "sim": 0.0
   

### Saving JSON as file WorldMapInput

In [437]:
with open('FINALVIZ.json', 'w') as outfile:
    json.dump(jsonBeautified, outfile)

## Further Analysis